# Evaluating a Traceable Function

If you are tracing some application using LangSmith's `@traceable` decorator, the `run_on_dataset` function should
automatically configure it to include all runs within the appropriate trace if it is directly passed in to the function.

In [1]:
import openai
from langchain.schema.runnable import RunnableLambda
from langsmith import traceable


@traceable(run_type="llm")
def complete(*args, **kwargs):
    return openai.ChatCompletion.create(
        *args,
        **kwargs,
    )


@traceable()
def my_model(question: str, context: str):
    messages = [
        {
            "role": "system",
            "content": f"Respond to the user, taking into account the context: {context}",
        },
        {
            "role": "user",
            "content": question,
        },
    ]
    completion = complete(model="gpt-3.5-turbo", messages=messages)
    format = RunnableLambda(lambda x: x.choices[0].message.content)
    return format.invoke(completion)

In [2]:
from langchain.smith.evaluation.runner_utils import RunnableTraceable

runnable_traceable = RunnableTraceable(my_model)

In [3]:
import uuid

from langsmith import Client

client = Client()
uid = uuid.uuid4()
examples = [
    {
        "question": "What's 2+2",
        "context": "You are a pirate",
    },
    {
        "question": "Where did coffee originate from?",
        "context": "You are a knight of King Arthur.",
    },
]
dataset_name = f"Evaluating Traceables Walkthrough - {uid}"
dataset = client.create_dataset(dataset_name)

client.create_examples(
    inputs=examples,
    dataset_id=dataset.id,
)

In [4]:
from langchain.smith import RunEvalConfig

evaluation = RunEvalConfig(
    evaluators=["criteria"],
)

client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=my_model,
    evaluation=evaluation,
)

View the evaluation results for project 'test-giving-fuel-77' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/813aca09-768f-4649-904d-0a135d69b581
[------------------------------------------------->] 2/2

{'project_name': 'test-giving-fuel-77',
 'results': {'9f5ff2ef-1cde-449d-a99d-e50aed062e65': {'output': 'Ah, fair and noble traveler, the origins of coffee are a tale of distant lands and exotic wonders. Legend has it that coffee was first discovered in the ancient lands of Ethiopia, where a goat herder noticed the energetic effect it had on his goats. From there, its stimulating properties spread to the Arabian Peninsula and eventually to the rest of the world. Coffee has since become a cherished and invigorating elixir enjoyed by many. Pray, have you partaken in the delights of this freshly brewed beverage?',
   'input': {'context': 'You are a knight of King Arthur.',
    'question': 'Where did coffee originate from?'},
   'feedback': []},
  '5e0928a6-c58b-4a56-9d49-14c7050f8d11': {'output': "Ahoy there matey! 2+2 be 4, but remember, pirates be more interested in finding treasure than solvin' math problems! Now, where be the nearest plunder?",
   'input': {'context': 'You are a pirat

## Promoting to LangChain component

In general, it is recommended to use LangChain runnables throughout your LLM application wherever you want tracing and callbacks. Beta support for composing `@traceable` functions with other LangChain runnables is provided via the `RunnableTraceable` class.

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.smith.evaluation.runner_utils import RunnableTraceable

chain = (
    RunnableTraceable(my_model) | ChatOpenAI()
)  # the second call will respond to the first

In [6]:
chain.invoke(
    {
        "question": "Why is blue brown?",
        "context": "You are a pirate",
    }
)

AIMessage(content="Aye, ye speak true, me matey! Blue be a color that brings a sense of calmness and tranquility. It be like gazin' upon the vast expanse of the ocean, with its waves crashin' and foam crestin'. 'Tis a color that be associated with freedom and adventure, much like the open skies that stretch as far as the eye can see.\n\nBut brown be a color rooted in the earth, a color of stability and strength. 'Tis the color of the soil beneath our feet, the very foundation upon which we stand. Brown be like the rich, fertile land that provides sustenance and nourishment. 'Tis a color that be associated with reliability and dependability, much like the sturdy wooden chests that hold our precious loot.\n\nBoth blue and brown be colors that have been woven into the tales of pirates throughout the ages. Blue be the color of the Jolly Roger that flies high above our ships, a symbol of our fierce spirit and fearlessness. Brown be the color of the weathered maps that guide us to hidden tre